In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
7,Pennsylvania COVID-19 Early Warning Monitoring...,https://www.governor.pa.gov/newsroom/pennsylva...,"Mon, 08 Feb 2021 18:18:06 GMT",2021-02-08 18:18:06+00:00
4,Covid: Over-70s can contact NHS for vaccine in...,https://www.bbc.com/news/health-55980053,"Mon, 08 Feb 2021 18:08:00 GMT",2021-02-08 18:08:00+00:00
5,New variants raise worry about Covid reinfecti...,https://www.nbcnews.com/health/health-news/new...,"Mon, 08 Feb 2021 17:51:00 GMT",2021-02-08 17:51:00+00:00
3,Weekly Operational Update on COVID-19 (8 Febru...,https://reliefweb.int/report/world/weekly-oper...,"Mon, 08 Feb 2021 17:33:00 GMT",2021-02-08 17:33:00+00:00
1,Rep. Ron Wright of Texas dies after battle wit...,https://www.latimes.com/obituaries/story/2021-...,"Mon, 08 Feb 2021 16:47:00 GMT",2021-02-08 16:47:00+00:00
6,Signs of self-sustained inflammatory circuits ...,https://www.nature.com/articles/d41586-021-002...,"Mon, 08 Feb 2021 16:25:34 GMT",2021-02-08 16:25:34+00:00
2,APH Releases COVID-19 Vaccine Distribution Das...,https://www.austintexas.gov/news/aph-releases-...,"Mon, 08 Feb 2021 16:07:04 GMT",2021-02-08 16:07:04+00:00
0,Health Department: 378 vaccine providers in al...,https://ldh.la.gov/index.cfm/newsroom/detail/6017,"Mon, 08 Feb 2021 16:03:03 GMT",2021-02-08 16:03:03+00:00
8,"In Dubai's COVID vaccine scramble, Sikhs serve...",https://abcnews.go.com/Health/wireStory/dubais...,"Mon, 08 Feb 2021 15:49:59 GMT",2021-02-08 15:49:59+00:00
9,South Africa halts AstraZeneca Covid vaccine r...,https://www.statnews.com/2021/02/07/south-afri...,"Mon, 08 Feb 2021 11:02:52 GMT",2021-02-08 11:02:52+00:00
